In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Live Orders with Holdings in LUSID

Attributes
----------
portfolio
recipe
quotes
instruments
holdings
orders
"""

toggle_code("Toggle Docstring")

# Incorporating live orders into your holdings view

In this example we demonstrate how to give front office users a view of intraday trading activity on top of their middle office IBOR. We set up a portfolio with multiple strategies and generate orders and allocations. We can show updated positions that account for these live, partially allocated orders.

To illustrate, below we will trade <i>Amazon, Inc.</i>. We'll create a set of limit, stop, and market orders, and show an updated view of the portfolio intraday.


## Imports

In [2]:
import lusid
import lusid.models as lm
import lusid.api as la
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)
from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os
from datetime import date,timedelta,datetime
from IPython.core.display import HTML

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 500)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

api_url = api_factory.api_client.configuration._base_path.replace("api","")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)
print ('LUSID Environment :', api_url)

LUSID Environment Initialised
API Version:  0.6.12078.0
LUSID Environment : https://steco.lusid.com/


In [3]:
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)
orders_api = api_factory.build(la.OrdersApi)
allocations_api = api_factory.build(lusid.api.AllocationsApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(la.AggregationApi)

Define scope, portfolio and some other variables used in the example:

In [4]:
scope = "holdings_with_live_orders"
portfolio_code = "us_long"
strategy_shk = "strategy"
date_today=date.today
pf_created_date = "2020-01-01T00:00:00+00:00"
recipe_code = "holdings_with_orders"


## Cleanup from prior runs of this code

In [5]:
myfilter= "portfolioId.code eq '" + portfolio_code + "' and portfolioId.scope eq '" + scope + "'"

orders_to_remove=orders_api.list_orders(filter= myfilter)

order_responses = {}
allocation_responses = {} 

for order in orders_to_remove.values:
    order_responses[order.id.code] = order

print(f"Removing orders: ",len(order_responses))

for order in order_responses:
            del_result = api_factory.build(la.OrdersApi).delete_order(scope,order)


Removing orders:  6


In [6]:
# Delete portfolio
print(f"Removing portfolio")
try:
    portfolios_api.delete_portfolio(scope, code=portfolio_code)
except:
    print('Portfolio does not exist yet.')

Removing portfolio


## Create a Portfolio

In [7]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code, "portfolio_name": name},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {
            "created": f"${pf_created_date}"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
    )

    succ, failed = format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])) 

In [8]:
create_portfolio(scope, portfolio_code, "Holdings + Live")

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


success  failed
0        1       0

## Load an Instrument Master

In [9]:
instr_df = pd.read_csv("data/live_orders_instruments.csv")
display(instr_df)

instrument_name client_internal currency          isin  \
0       Amazon_Nasdaq_AMZN    imd_34634534      USD  US0231351067   
1       Anglian_Water_40LV    imd_13579246      USD  XS0089553282   
2  ICE_Silver_5000oz_Dec21    imd_12457801      USD           NaN   

           figi exchange_code          country_issue ticker market_sector  \
0  BBG000BVPXP1            UN  united_states_america   AMZN        equity   
1           NaN            GB         united_kingdom    NaN  fixed_income   
2           NaN            GB         united_kingdom  ZIZ21     commodity   

  security_type  coupon  
0  common_stock     NaN  
1          bond   6.625  
2        future     NaN

In [10]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}

In [11]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"]
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


success  failed  errors
0        3       0       0

## Create a recipe

In [12]:
configuration_recipe = lm.ConfigurationRecipe(
    scope,
    recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        )
    ),
    pricing=lm.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
    ),
)

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

## Set initial holdings in the parent portfolio

In [13]:
hldgs_df = pd.read_csv("data/initial_amazon_holdings.csv")
display(hldgs_df)

instrument_name client_internal          isin          figi  quantity  \
0  Amazon_Nasdaq_AMZN    imd_34634534  US0231351067  BBG000BVPXP1      5000   

   unit_cost  total_cost currency  
0        165      825000      USD

In [14]:
holdings_mapping = {
    "required":{
        "code": f"${portfolio_code}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity"
    },
    "identifier_mapping": {
        "ClientInternal": "client_internal",
    },
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost"
    }
}

In [15]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    file_type="holdings"
)
succ, failed = format_holdings_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


success  failed  errors
0        1       0       0

### Upsert Quotes
Load some sample data for our instrument. We're uploading a quote for today, as market orders are converted to holdings by assuming that they're filled at the instant we make the query and take the latest market price for it.

In [16]:
quotes_df = pd.read_excel("data/quotes.xlsx")
quotes_df['date']=date.today()
quotes_df.head()

date ticker                     name          figi  \
0  2023-10-18   AMZN            Amazon Nasdaq  BBG000BVPXP1   
1  2023-10-18    NaN       Anglian Water 40LV           NaN   
2  2023-10-18  ZIZ21  ICE Silver 5000oz Dec21           NaN   

                          Sector  open_price  close_price client_internal  
0  Equity Investment Instruments      111.00        111.5    imd_34634534  
1                   Fixed Income      110.93        112.2    imd_13579246  
2                    Commodities       24.30         25.4    imd_12457801

In [17]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$ClientInternal",
    "quote_id.effective_at": "date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "client_internal",
    "metric_value.unit": "$USD",
}

In [18]:
quotes_mapping["quote_id.quote_series_id.field"] ="$mid"
quotes_mapping["metric_value.value"] = "close_price"

result = load_from_data_frame(
    api_factory = api_factory,
    scope=scope,
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes"
)
succ, failed, errors = format_quotes_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]))

/opt/conda/lib/python3.11/site-packages/lusidtools/cocoon/cocoon.py:1720: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_frame = data_frame.applymap(cocoon.utilities.convert_cell_value_to_string)


success  failed  errors
0        3       0       0

## Check the initial portfolio holdings
We see just a single, material holding in the instrument.

In [19]:
executed_holdings = transaction_portfolios_api.get_holdings_with_orders(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    recipe_id_scope=scope,recipe_id_code=recipe_code)

response_df=lusid_response_to_data_frame(executed_holdings, rename_properties=True)
response_df[["instrument_uid", "holding_type", "units", "cost.amount", "cost.currency"]].style.format({"units":"{:20,.0f}","cost.amount": "{:20,.2f}"})

## Post orders into LUSID

In this section, we post some [orders](https://support.finbourne.com/how-does-lusid-support-the-trade-lifecycle) into LUSID. We have a number of orders for Amazon stock to demonstrate the processing of market, limit, and stop orders. In addtion we have an order for a corporate bond.

In [20]:
orders_df = pd.read_csv('data/live_orders.csv')
orders_df

portfolio          instrument_name client_internal          isin  \
0   us_long       Amazon_Nasdaq_AMZN    imd_34634534  US0231351067   
1   us_long       Amazon_Nasdaq_AMZN    imd_34634534  US0231351067   
2   us_long       Amazon_Nasdaq_AMZN    imd_34634534  US0231351067   
3   us_long       Amazon_Nasdaq_AMZN    imd_34634534  US0231351067   
4   us_long       Anglian_Water_40LV    imd_13579246  XS0089553282   
5   us_long  ICE_Silver_5000oz_Dec21    imd_12457801           NaN   

           figi  quantity  price currency order_id  side    type state  \
0  BBG000BVPXP1      1000  100.0      USD   ORD001  sell   limit   new   
1  BBG000BVPXP1      1000   95.0      USD   ORD002   buy   limit   new   
2  BBG000BVPXP1      3000    NaN      USD   ORD003   buy    stop   new   
3  BBG000BVPXP1      1000    NaN      USD   ORD004   buy  market   new   
4           NaN       500    NaN      USD   ORD005   buy  market   new   
5           NaN      5000    NaN      USD   ORD006   buy  market   new   

   limit_price limit_currency  stop_price stop_currency  
0       100.50            USD         NaN           NaN  
1        96.25            USD         NaN           NaN  
2          NaN            NaN        94.0           USD  
3          NaN            NaN         NaN           NaN  
4          NaN            NaN         NaN           NaN  
5          NaN            NaN         NaN           NaN

In [21]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)
responses = []

for index, order in orders_df.iterrows():
    
    request = lm.OrderRequest(
            id=lm.ResourceId(
                scope=scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': order['client_internal']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=scope,
                code=portfolio_code
            ),
            state=order['state'],
            type=order['type'],
            price=lm.CurrencyAndAmount(
                        amount=0 if pd.isna(order['price']) else order['price'],
                        currency=order['currency']
            ),

            limit_price=lm.CurrencyAndAmount(
                        amount=order['limit_price'],
                        currency=order['limit_currency']
            )            
            if not pd.isna(order['limit_price']) and not pd.isna(order['limit_currency']) else None,
            
            stop_price=lm.CurrencyAndAmount(
                        amount=order['stop_price'],
                        currency=order['stop_currency']
            )
            if not pd.isna(order['stop_price']) and not pd.isna(order['stop_currency']) else None
    )
    
    request=lm.OrderSetRequest(
        order_requests=[request]           
    )

    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=request
    )
    
    responses.append(response.values[0])

attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/ClientInternal'],
             o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount,o.price.currency,
             o.limit_price.amount if o.limit_price is not None else "N/A",
             o.limit_price.currency if o.limit_price is not None else "N/A") for o in responses]

pd.DataFrame(attributes, columns=['order_id','client_internal','lusid_instrument_id','side','type',
                                  'state','quantity','price','currency','lim px',
                                  'lim ccy']).style.format({"quantity":"{:20,.0f}","price": "{:20,.2f}"})

### Post allocations into LUSID
In this section, we post some allocations into LUSID. These allocations represent full or partial allocations against a subset of the originating orders.

In [22]:
allocations_df = pd.read_csv('data/amazon_allocations.csv')
allocations_df

portfolio     instrument_name client_internal          isin          figi  \
0   us_long  Amazon_Nasdaq_AMZN    imd_34634534  US0231351067  BBG000BVPXP1   

   quantity  price currency        allocation_id originating_order  \
0       777  102.7      USD  ALLOC001-for-ORD004            ORD004   

                state side    type  
0  partial allocation  buy  market

In [23]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)
responses = []

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    request = lm.AllocationRequest(
            id=lm.ResourceId(
                scope=scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id = lm.ResourceId(
                scope=scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            side=allocation['side'],
            instrument_identifiers={
                'Instrument/default/ClientInternal': allocation['client_internal']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=scope,
                code=portfolio_code
            ),
            state=allocation['state'],
            type=allocation['type'],
            price=lm.CurrencyAndAmount(
                        amount=allocation['price'],
                        currency=allocation['currency']))
    
    request=lm.AllocationSetRequest(
        allocation_requests=[request]           
    )

    response = allocations_api.upsert_allocations(
        allocation_set_request=request
    )
    
    responses.append(response.values[0])

attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/ClientInternal'],o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount) for o in responses]
pd.DataFrame(attributes, columns=['allocation_id','client_internal','lusid_instrument_id','side','type','state','quantity','price']).style.format({"quantity":"{:20,.0f}","price": "${:20,.2f}"})

## Check the portfolio holdings with orders
Finally we can ask LUSID to give us a live holdings view that includes contributions from outstanding orders of market, limit, or stop type.

Key:
- P: Position
- O, OC: Order (O) and OrderCash (OC) holdings for the security and cash commitments of each outstanding order. 
- L, LC: Partially and fully-allocated orders will have Allocation (L) and AllocationCash (LC) holdings to represent allocations against them, and have their quantities suitably reduced.

In [24]:
executed_holdings = transaction_portfolios_api.get_holdings_with_orders(
    scope=scope, 
    code=portfolio_code, 
    property_keys=["Instrument/default/Name"], 
    recipe_id_scope=scope, 
    recipe_id_code=recipe_code)

response_df = lusid_response_to_data_frame(executed_holdings, rename_properties=True)

response_df.rename(columns = {'transaction.transaction_price.price':'price','Name(default-Properties)':'instrument','cost.amount':'cost','cost.currency':'ccy','cost_portfolio_ccy.amount':'pfolio_cost'}, inplace = True)
response_df['transaction.transaction_id'] = response_df['transaction.transaction_id'].fillna('')

ordered=response_df.sort_values('transaction.transaction_id')
display(ordered[["instrument", "holding_type", "units", "cost", "ccy", "pfolio_cost",  "price"]].style.format({"price":"{:20,.2f}","units":"{:20,.0f}","cost": "{:20,.2f}","pfolio_cost": "{:20,.2f}"}))

In [25]:
# Setup the aggregation request 
aggregation_request = lm.ValuationRequest(
        recipe_id = lm.ResourceId(
            scope = scope,
            code = recipe_code
        ),
        metrics = [
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Valuation/PV", "Proportion"),
            lm.AggregateSpec("Valuation/PV", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Holding/HoldingType", "Value"),
        ],
        # choose the valuation date for the request - set using effectiveAt
        valuation_schedule=lm.ValuationSchedule(effective_at=date.today().isoformat()),
        portfolio_entity_ids = [lm.PortfolioEntityId(
                                                        scope = scope,
                                                        code = portfolio_code,
                                                        portfolio_entity_type="SinglePortfolio" 
            )],
        include_order_flow=lm.OrderFlowConfiguration("OrdersAndAllocations")
        )
    
# Pull the data aggregation by passing the effectiveAt date
aggregation = aggregation_api.get_valuation(valuation_request=aggregation_request)
df = pd.DataFrame(aggregation.data)

df

Instrument/default/Name  Proportion(Valuation/PV)  Valuation/PV  \
0        Amazon_Nasdaq_AMZN                  0.897619      557500.0   
1        Amazon_Nasdaq_AMZN                  0.040034       24864.5   
2                       USD                 -0.040034      -24864.5   
3        Anglian_Water_40LV                  0.090325       56100.0   
4                       USD                 -0.090325      -56100.0   
5   ICE_Silver_5000oz_Dec21                  0.204480      127000.0   
6                       USD                 -0.204480     -127000.0   
7        Amazon_Nasdaq_AMZN                 -0.179524     -111500.0   
8                       USD                  0.161813      100500.0   
9        Amazon_Nasdaq_AMZN                  0.179524      111500.0   
10                      USD                 -0.154970      -96250.0   
11       Amazon_Nasdaq_AMZN                  0.538571      334500.0   
12                      USD                 -0.454042     -282000.0   
13       Amazon_Nasdaq_AMZN                  0.139490       86635.5   
14                      USD                 -0.128481      -79797.9   

    Holding/default/Units Holding/HoldingType  
0                  5000.0                   P  
1                   223.0                   O  
2                -24864.5                  OC  
3                   500.0                   O  
4                -56100.0                  OC  
5                  5000.0                   O  
6               -127000.0                  OC  
7                 -1000.0                   O  
8                100500.0                  OC  
9                  1000.0                   O  
10               -96250.0                  OC  
11                 3000.0                   O  
12              -282000.0                  OC  
13                  777.0                   L  
14               -79797.9                  LC

In [26]:
display(HTML('<h2>Links</h2>'))

display(HTML(f'<a href="{api_url}app/dashboard/holdings?scope={scope}&code={portfolio_code}&entityType=Portfolio&withOrders=true&withAllocations=true" target="_blank">See holdings including open orders</a>'))
display(HTML(f'(If this does not return the data you may need to set the <a href="https://steco.lusid.com/app/system-settings/default-settings/recipe-settings">Default recipe</a> to the recipe holdings_with_live_orders/holdings_with_orders'))

display(HTML('<br />'))
        
display(HTML(f'<a href="{api_url}app/dashboard/valuations?scope={scope}&code={portfolio_code}&entityType=Portfolio&recipeScope={scope}&{recipe_code}=market-value&taxLots=false&withOrders=true&withAllocations=true" target="_blank">See valuation including open orders</a>'))